In [20]:
import pandas as pd
import numpy as np
from talib import abstract

pd.set_option('display.max_rows', 10000)

In [37]:
import time

def EMA_dataframe(pair, days):
    cool = time.perf_counter()
    currency = pd.read_csv("../data/external/exchange_rates/{}_M1.csv".format(pair))
    currency.columns = ["Time", "Volume", "Open", "Close", "High", "Low"]
    currency["Time"] = currency["Time"].transform(lambda time : time[0:14])
    
    start, end = find_interval_index(currency)
    
    window_size = 24*60*days

    daily_ema = np.empty(end - start + 1, dtype="float")
    date_ema = np.empty(end - start + 1, dtype="object")
    window_size = 24*60*days
    for i in range(start, end + 1):
        if i - start < window_size:
            daily_ema[i - start] = currency["Close"][i - window_size:i + 1].mean()
        else:
            daily_ema[i - start] = calculate_EMA(currency["Close"][i - 1], days, daily_ema[i - start - window_size])
        date_ema[i - start] = currency["Time"][i - 1]
            
    daily_ema = pd.DataFrame({"Time": date_ema, "EMA_{}".format(days): daily_ema})
    daily_ema = daily_ema.merge(currency, how="left", on="Time")

    time_elapsed = time.perf_counter() - cool
    
    return daily_ema

def calculate_EMA(close, days, previous_ema):
    return previous_ema + (2/(1 + days))*(close - previous_ema)

def EMA_merge(days_1, days_2, currency):
    ema_1 = EMA_dataframe(currency, days_1)
    ema_2 = EMA_dataframe(currency, days_2)
    return ema_1.merge(ema_2, how="outer", on=["Time", "Volume", "Open", "Close", "High", "Low"])

def find_interval_index(currency):
    start = 0
    end = 0
    index = 0
    for date in currency["DateTime"]:
        if start == 0 and date[:4] == "2018":
            start = index
        elif end == 0 and date[:4] == "2021":
            end = index
            break;
        index += 1
    return start, end

In [5]:
def AD_Dataframe(pair):
    cool = time.perf_counter()
    currency = pd.read_csv("../data/external/exchange_rates/{}_M1.csv".format(pair))
    currency.columns = ["Time", "Volume", "Open", "Close", "High", "Low"]

    start, end = find_interval_index(currency)

    ad_index = np.empty(end - start + 1, dtype="float")
    ad_date = np.empty(end - start + 1, dtype="object")
    window_size = 24*60

    for i in range(start, end + 1):
        close_mean = currency["Close"][i - window_size:i + 1].mean()
        high = currency["High"][i - window_size:i + 1].max()
        low = currency["Low"][i - window_size:i + 1].min()
        volume = currency["Volume"][i - window_size:i + 1].sum()
        
        ad_index[i - start] = calculate_AD(close_mean, high, low, volume)
        ad_date[i - start] = currency["Time"][i - 1][0:14]

    ad_index = pd.DataFrame({"Time": ad_date, "A/D Index": ad_index})
    time_elapsed = time.perf_counter() - cool
    return ad_index

def calculate_AD(close, high, low, volume):
    if high - low == 0:
        return 0
    else:
        return volume*((close - low) - (high - close))/(high - low)

In [17]:
from datetime import datetime

def convert_date(exchange):
    exchange = exchange.rename(columns={'DateTime': 'Time'})
    exchange["Time"] = pd.to_datetime(exchange["Time"], format="%Y-%m-%d %H:%M:%S")
    return exchange


In [7]:
def RSI_dataframe(pair):
    cool = time.perf_counter()
    currency = pd.read_csv("../data/external/exchange_rates/{}_M1.csv".format(pair))
    currency.columns = ["Time", "Volume", "Open", "Close", "High", "Low"]

    start, end = find_interval_index(currency)
    
    period_window = 60*24*14
    day_window = 60*24
    rsi_index = np.empty(end - start + 1, dtype="float")
    rsi_date = np.empty(end - start + 1, dtype="object")
    rsi_gain = np.empty(end - start + 1, dtype="float")
    rsi_loss = np.empty(end - start + 1, dtype="float")
    
    for i in range(start, end + 1):
        if i - start < period_window - 1:
            gain = 0
            loss = 0
            for j in range(14):
                curr_open = currency["Open"][i - day_window*(j + 1)]
                curr_close = currency["Close"][i - day_window*j - 1]
                change = curr_close - curr_open
                if change < 0:
                    loss -= change
                else:
                    gain += change
            rsi_gain[i - start] = gain/14
            rsi_loss[i - start] = loss/14
            rsi_index[i - start] = calculate_RSI(gain, loss)
        else:
            gain = 0
            loss = 0
            for j in range(14):
                curr_open = currency["Open"][i - day_window*(j + 1)]
                curr_close = currency["Close"][i - day_window*j - 1]
                change = curr_close - curr_open
                if change < 0:
                    loss -= change
                else:
                    gain += change
            curr_gain = gain/14
            curr_loss = loss/14
            rsi_gain[i - start] = curr_gain
            rsi_loss[i - start] = curr_loss
            prv_gain = rsi_gain[i - start - period_window + 1:i - start].mean()
            prv_loss = rsi_loss[i - start - period_window + 1:i - start].mean()
            rsi_index[i - start] = calculate_RSI(curr_gain, curr_loss, prv_gain, prv_loss)
            
        rsi_date[i - start] = currency["Time"][i - 1][0:14]
    
    rsi_index = pd.DataFrame({"Time": rsi_date, "RSI": rsi_index})
    time_elapsed = time.perf_counter() - cool
    return rsi_index
    

def calculate_RSI(curr_gain, curr_loss, prv_avg_gain=None, prv_avg_loss=None): 
    if not prv_avg_gain and not prv_avg_loss:
        avg_gain = curr_gain/14
        avg_loss = curr_loss/14
        if avg_loss == 0:
            return 100
        rs = avg_gain/avg_loss
        if rs == -1:
            return 0
        return 100 - 100/(1 + rs)
    else:
        avg_gain = (prv_avg_gain*13 + curr_gain)/14
        avg_loss = (prv_avg_loss*13 + curr_loss)/14
        if avg_loss == 0:
            return 100
        rs = avg_gain/avg_loss
        if rs == -1:
            return 0
        return 100 - 100/(1 + rs)

In [59]:
"""eur_usd_rsi = RSI_dataframe("EURUSD")
eur_usd_ema = EMA_merge(10, 100, "EURUSD")
eur_usd_ad = AD_Dataframe("EURUSD")
eur_usd_exchange = eur_usd_rsi.merge(eur_usd_ema, how="outer", on="Time")
eur_usd_exchange = eur_usd_exchange.merge(eur_usd_ad, how="outer", on="Time")"""
currency = pd.read_csv("../data/external/exchange_rates/{}_M1.csv".format("EURUSD"))
start, end = find_interval_index(currency)
currency = convert_date(currency)

In [60]:
currency['EMA_10'] = pd.DataFrame(abstract.EMA(currency['Close'], timeperiod=10))
currency['EMA_100'] = pd.DataFrame(abstract.EMA(currency['Close'], timeperiod=100))
currency['RSI'] = pd.DataFrame(abstract.RSI(currency['Close'], timeperiod=14))
currency['A/D Index'] = pd.DataFrame(abstract.AD(currency['High'], currency['Low'], currency['Close'], currency['Volume']))
currency

,Time,Volume,Open,Close,High,Low,EMA_10,EMA_100,RSI,A/D Index
0,2017-01-01 22:00:00,2,1.05148,1.05153,1.05153,1.05148,NaN,NaN,NaN,2.000000e+00
1,2017-01-01 22:01:00,2,1.05153,1.05153,1.05153,1.05153,NaN,NaN,NaN,2.000000e+00
2,2017-01-01 22:02:00,9,1.05170,1.05175,1.05175,1.05170,NaN,NaN,NaN,1.100000e+01
3,2017-01-01 22:03:00,1,1.05174,1.05175,1.05175,1.05174,NaN,NaN,NaN,1.200000e+01
4,2017-01-01 22:08:00,6,1.05170,1.05170,1.05170,1.05170,NaN,NaN,NaN,1.200000e+01
...,...,...,...,...,...,...,...,...,...,...
1523482,2021-02-01 23:55:00,6,1.20678,1.20677,1.20678,1.20677,1.206761,1.206443,59.055022,-7.529389e+06
1523483,2021-02-01 23:56:00,34,1.20677,1.20677,1.20677,1.20674,1.206763,1.206449,59.055022,-7.529355e+06
1523484,2021-02-01 23:57:00,31,1.20679,1.20681,1.20681,1.20679,1.206772,1.206456,62.443562,-7.529324e+06
1523485,2021-02-01 23:58:00,60,1.20681,1.20678,1.20683,1.20678,1.206773,1.206463,58.531146,-7.529384e+06


In [61]:
print(start, end)

373106 1493278


In [62]:
currency = currency.loc[start:end - 1, :]
currency

,Time,Volume,Open,Close,High,Low,EMA_10,EMA_100,RSI,A/D Index
373106,2018-01-01 22:00:00,13,1.20102,1.20100,1.20102,1.20097,1.199956,1.200065,77.978187,-1.550852e+06
373107,2018-01-01 22:01:00,12,1.20099,1.20007,1.20099,1.20007,1.199976,1.200065,54.213030,-1.550864e+06
373108,2018-01-01 22:02:00,5,1.20015,1.20025,1.20025,1.20015,1.200026,1.200069,56.947897,-1.550859e+06
373109,2018-01-01 22:03:00,15,1.20024,1.20029,1.20050,1.20024,1.200074,1.200073,57.554629,-1.550868e+06
373110,2018-01-01 22:04:00,8,1.20032,1.20032,1.20035,1.20031,1.200119,1.200078,58.032337,-1.550872e+06
...,...,...,...,...,...,...,...,...,...,...
1493273,2020-12-31 21:55:00,255,1.22175,1.22174,1.22180,1.22170,1.221732,1.221836,43.865936,-7.348274e+06
1493274,2020-12-31 21:56:00,193,1.22176,1.22165,1.22179,1.22160,1.221717,1.221832,38.963768,-7.348365e+06
1493275,2020-12-31 21:57:00,77,1.22166,1.22164,1.22167,1.22153,1.221703,1.221828,38.449612,-7.348321e+06
1493276,2020-12-31 21:58:00,246,1.22163,1.22153,1.22168,1.22143,1.221672,1.221822,33.251739,-7.348370e+06


In [71]:
def indicators(currency):
    currency = pd.read_csv("../data/external/exchange_rates/{}_M1.csv".format(currency))
    start, end = find_interval_index(currency)
    currency = convert_date(currency)
    
    currency['EMA_10'] = pd.DataFrame(abstract.EMA(currency['Close'], timeperiod=10))
    currency['EMA_100'] = pd.DataFrame(abstract.EMA(currency['Close'], timeperiod=100))
    currency['RSI'] = pd.DataFrame(abstract.RSI(currency['Close'], timeperiod=14))
    currency['A/D Index'] = pd.DataFrame(abstract.AD(currency['High'], currency['Low'], currency['Close'], currency['Volume']))
    currency['A/D Index'] = currency['A/D Index'] - currency['A/D Index'].shift(1)
    currency = currency.loc[start:end - 1, :]
    currency = stationary_log_returns(currency)
    
    currency = currency.iloc[1:]
    
    return currency
    
    
def stationary_log_returns(pair):
    pair = pair.copy()
    pair["Close"] = np.log(pair["Close"]/pair["Close"].shift(1))
    pair["Open"] = np.log(pair["Open"]/pair["Open"].shift(1))
    pair["High"] = np.log(pair["High"]/pair["High"].shift(1))
    pair["Low"] = np.log(pair["Low"]/pair["Low"].shift(1))
    pair["EMA_10"] = np.log(pair["EMA_10"]/pair["EMA_10"].shift(1))
    pair["EMA_100"] = np.log(pair["EMA_100"]/pair["EMA_100"].shift(1))
    return pair

In [72]:
cool = indicators("EURUSD")
cool

,Time,Volume,Open,Close,High,Low,EMA_10,EMA_100,RSI,A/D Index
373107,2018-01-01 22:01:00,12,-0.000025,-0.000775,-0.000025,-0.000750,0.000017,8.201449e-08,54.213030,-12.000000
373108,2018-01-01 22:02:00,5,-0.000700,0.000150,-0.000616,0.000067,0.000041,3.050522e-06,56.947897,5.000000
373109,2018-01-01 22:03:00,15,0.000075,0.000033,0.000208,0.000075,0.000040,3.650132e-06,57.554629,-9.230769
373110,2018-01-01 22:04:00,8,0.000067,0.000025,-0.000125,0.000058,0.000037,4.072857e-06,58.032337,-4.000000
373111,2018-01-01 22:05:00,24,-0.000008,0.000167,0.000142,0.000000,0.000061,7.292287e-06,61.169916,24.000000
...,...,...,...,...,...,...,...,...,...,...
1493273,2020-12-31 21:55:00,255,0.000131,-0.000008,-0.000016,0.000139,0.000001,-1.579965e-06,43.865936,-51.000000
1493274,2020-12-31 21:56:00,193,0.000008,-0.000074,-0.000008,-0.000082,-0.000012,-3.007291e-06,38.963768,-91.421053
1493275,2020-12-31 21:57:00,77,-0.000082,-0.000008,-0.000098,-0.000057,-0.000012,-3.109818e-06,38.449612,44.000000
1493276,2020-12-31 21:58:00,246,-0.000025,-0.000090,0.000008,-0.000082,-0.000026,-4.831007e-06,33.251739,-49.200000


In [73]:
cool.to_csv("../data/processed/exchange_rate/EURUSD_exchange.csv", index=False)